# Ansbile playbook

## 1. Simple playbook

### 1.1. Ping test

See [ping.yml](./ping.yml)

#### 1.1.1. Ping remote server as current user

- Single server

In [ ]:
ansible-playbook ping.yml --tags "G1"

- Multi-servers

In [ ]:
ansible-playbook ping.yml --tags "G2"

#### 1.1.2. Ping remote server as become user

- `become`: Run as root user
```yml
tasks:
    - name: Run as root user
      become: True
```
- `-e "@password.yml"`: Read external arguments from encrypted file `password.yml`
- `--vault-id vault-id`: Read password from `vault-id` file for encrypted file

In [ ]:
ansible-playbook ping.yml --tags "G3" --vault-id vault-id -e "@password.yml"

### 1.2. Ansible variables

See [vars.yml](./vars.yml)

#### 1.2.1. Ansible builtin variables

- Built in variable prefix by `ansible_`
- `gather_facts`: `True` if need gather the ansible variables from remote server

In [ ]:
ansible-playbook vars.yml --tags "G1"

#### 1.2.2. User define variables

- `vars`: define variables
```yml
vars:
    var1: value1
    var2: value2
```
- Without external args

In [ ]:
ansible-playbook vars.yml --tags "G2"

- With external args
    - `-e`: external arguments, if will override the same argument defined in playbook

In [ ]:
ansible-playbook vars.yml -e "user_name=Emma user_age=32" --tags "G2"

#### 1.2.3. Include variables file


- `include_vars`: Load variables from given file, the file format can be `json` or `ymal`

```yml
tasks:
  - name: Include vars from file
    include_vars: variables_file.yml
```

In [ ]:
ansible-playbook vars.yml --tags "G3"

#### 1.2.4. Environment variables

- `environment`: Set environment variables on remote server for current process

```yml
  environment:
    env_var1: value1
    env_var2: value2
```

- `ansible_env`: The ansible facts variable, an dictionary that include global environment variables

```yml
  tasks:
    - name: Run when value of remote environment vairable "RUNNING_STATUS" is "OK"
      when: ansible_env.RUNNING_STATUS == "OK"
      # or ansible_env["RUNNING_STATUS"] == "OK"
      
    - name: Show value of remote environment variable "NAME"
      debug: 
        msg: "The value is: {{ ansible_env.NAME }}"
```

- `lookup('env', <name>)`: Get special environment variables by name on local machine

```yml
  tasks:
    - name: Show value of local environment variable "HOST"
      debug: 
        msg: "The value is: {{ lookup('env', 'HOST') }}"
      when: (lookup('env', 'HOST') is defined)
```

In [ ]:
VAR_MSG="HELLO WORLD" ansible-playbook vars.yml --tags "G4"

#### 1.2.5. Show ansible variables and facts variables

- `ansible_facts`: A dictionary that include all facts variables fetched from remote server, the `gather_facts` field must be `True`

```yml
tasks:
  - name: Show ansible facts variables
    debug:
      msg: 
        - "Architecture: {{ ansible_facts.architecture }}"
```

In [ ]:
ansible-playbook vars.yml --tags "G5"

#### 1.2.6. Set variable by task

- `set_fact`: `set_fact` module can set variable in task

```yml
tasks:
  - name: Set variables "a" and "b"
    set_fact:
      a: 100
      b: 200
```

In [ ]:
ansible-playbook vars.yml --tags "G6"

### 1.3. Conditionals

See [conditional.yml](./conditional.yml)

#### 1.3.1. Basic conditionals with `when`

- `when <conditional>`: Task execute when value of conditional statement is `True`

```yml
tasks:
  - name: Run shell command if target distribution is "Debian"
    shell: uname -sa
    when: ansible_distribution == "Debian"
```

In [ ]:
ansible-playbook conditional.yml --tags "G1"

#### 1.3.2. Conditionals based on `ansible_facts`

```yml
tasks:
  - name: Run shell command if target distribution is "Debian" and version ge than 10
    shell: uname -sa
    when: >
      ansible_facts.distribution == "Debian" and 
      (ansible_facts.distribution_major_version | int) >= 10
```

In [ ]:
ansible-playbook conditional.yml --tags "G2"

#### 1.3.3. Conditions based on registered variables

- `register`: Register a variable to save result of task

```yaml
tasks:
  - name: Get "uname -sa" result as register variable
    shell: uname -sa
    register: uname_result

  - name: Show "stdout" propertiy of register variable
    debug:
      msg: "{{ uname_result.stdout }}"
    when: (uname_result.stdout | length) > 0
```

In [ ]:
ansible-playbook conditional.yml --tags "G3"

#### 1.3.4. Conditions based on operation result

- `skipped`: Registered variable is marked as 'skipped', the task was skipped

```yaml
tasks:
  - name: "Skipping" task
    command: /usr/bin/true
    register: cmd_result
    when: False
  
  - name: Execute if lastest task is skipped
    debug: 
      msg: "command {{ cmd }} is skipped"
    when: (cmd_result is skipped)
```

In [ ]:
ansible-playbook conditional.yml --tags "G4"

- `succeeded`: Registered variable is marked as 'succeeded', the task worked succsessful

```yaml
tasks:
  - name: "Skipping" task
    command: /usr/bin/false
    register: cmd_result

  - name: Show "{{ cmd }}" result if it succeeded
    debug: 
      msg: "{{ cmd_result.msg }}"
    when: (cmd_result is succeeded)
```

In [ ]:
CMD="ls -al" ansible-playbook conditional.yml --tags "G4"

- `failed`: Registered variable is marked as 'failed', the task finish with error

```yaml
tasks:
  - name: "Skipping" task
    command: /usr/bin/false
    register: cmd_result

  - name: Show "{{ cmd }}" result if it failed
    debug: 
      msg: "{{ cmd_result.msg }}"
    when: (cmd_result is failed)
```

In [ ]:
CMD="bad_command" ansible-playbook conditional.yml --tags "G4"

### 1.4. Loop

See [loop.yml](./loop.yml)

#### 1.4.1. Simple loop

- `with_items <collection>`

```yml
tasks:
  - name: Loop by "with_items"
    debug:
      msg: "item is: {{ item }}"
    with_items:
      - a
      - b
      - c
      - d
```
    Or 
```yml
vars:
  items: ['a', 'b', 'c']
tasks:
  - name: Loop by "with_items"
    debug:
      msg: "item is: {{ item }}"
    with_items: "{{ items }}"
```

In [ ]:
ansible-playbook loop.yml --tags "G1"

#### 1.4.2. Nested loop

- `with_nested: [<collection1>, <collection2>, ...]`

```yml
tasks:
  - name: Loop by "with_nested"
    debug:
      msg: "item[0] is: {{ item[0] }} and item[1] is: {{ item[1] }}"
    with_nested:
      - ['a', 'b', 'c']
      - ['x', 'y']
```
    Or
```yml
vars:
  list1: ['a', 'b', 'c']
  list2: ['x', 'y']
tasks:
  - name: Loop by "with_nested"
    debug:
      msg: "item[0] is: {{ item[0] }} and item[1] is: {{ item[1] }}"
    with_nested:
      - "{{ list1 }}"
      - "{{ list2 }}"
```

In [ ]:
ansible-playbook loop.yml --tags "G2"

#### 1.4.3. Loop with task results

- `with_items <register>.stdout_lines`

```yml
tasks:
  - name: List files in root
    shell:
      cmd: ls -al
      chdir: /
    register: ls_result
      
  - name: Loop by task result
    debug:
      msg: "Item is: {{ item }}"
    with_items: "{{ ls_result.stdout_lines }}"
```

In [ ]:
ansible-playbook loop.yml --tags "G3"

#### 1.4.4. Loop in dictionary

- `with_dict <dictionary>`

```yml
vars:
  map:
    a: 100
    b: 200
tasks:
  - name: Loop in dictionary with key and value
    debug:
      msg: "{{ item.key }}: {{ item.value }}"
    with_dict: "{{ map }}"
```

In [ ]:
ansible-playbook loop.yml --tags "G4"

#### 1.4.5. Loop in file list

- `with_fileglob [<glob1>, <glob2>, ...]`

```yml
tasks:
  - name: Loop in file list
    debug:
      msg: "{{ item }}"
    with_fileglob: 
      - ./*.yml
```

In [ ]:
ansible-playbook loop.yml --tags "G5"

### 1.5. Block

See [block.yml](./block.yml)

#### 1.5.1. Run task group by blocks

In [ ]:
ansible-playbook block.yml --tags "G1"

#### 1.5.2. Error handle by blocks

In [ ]:
ansible-playbook block.yml --tags "G2"

#### 1.5.3. Error aborting

- When each remote server raise error, the whole play abort on all servers

In [ ]:
ansible-playbook block.yml --tags "G3"

### 1.6. Error handle

See [error_handle.yml](./error_handle.yml)

#### 1.6.1. Ignoring failed tasks

- `ignore_errors`
```yml
tasks:
    - name: Ignoring failed tasks
      command: /bin/false
      ignore_errors: True
```

In [ ]:
ansible-playbook error_handle.yml --tags "G1"

#### 1.6.2. Ignoring unreachable host errors

In [ ]:
ansible-playbook error_handle.yml --tags "G2"

#### 1.6.3. Defining failure

In [ ]:
ansible-playbook error_handle.yml --tags "G3"

#### 1.6.4. Defining “changed”

In [ ]:
ansible-playbook error_handle.yml --tags "G4"

#### 1.6.5. Ensuring success for shell command

In [ ]:
ansible-playbook error_handle.yml --tags "G5"

#### 1.6.6. Aborting a play on all hosts

- `any_errors_fatal`

In [ ]:
ansible-playbook error_handle.yml --tags "G6-1"

- Setting a maximum failure percentage

In [ ]:
ansible-playbook error_handle.yml --tags "G6-2"